In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import re
import string
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-05-04 20:22:03.130260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 20:22:03.130352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 20:22:03.231106: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")

In [3]:
categories_l = len(df['Category'].unique())
cats = list(df['Category'].unique())

In [4]:
cats

['HR',
 'DESIGNER',
 'INFORMATION-TECHNOLOGY',
 'TEACHER',
 'ADVOCATE',
 'BUSINESS-DEVELOPMENT',
 'HEALTHCARE',
 'FITNESS',
 'AGRICULTURE',
 'BPO',
 'SALES',
 'CONSULTANT',
 'DIGITAL-MEDIA',
 'AUTOMOBILE',
 'CHEF',
 'FINANCE',
 'APPAREL',
 'ENGINEERING',
 'ACCOUNTANT',
 'CONSTRUCTION',
 'PUBLIC-RELATIONS',
 'BANKING',
 'ARTS',
 'AVIATION']

In [5]:
df

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [6]:
df = df.sample(frac=1)

In [7]:
df

,ID,Resume_str,Resume_html,Category
1517,76454959,FINANCE DIRECTOR Executive Prof...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE
1603,50324968,STYLIST Objective An interns...,"<div class=""fontsize fontface vmargins hmargin...",APPAREL
2328,73030450,SENIOR NETWORK SECURITY ENGINEER ...,"<div class=""fontsize fontface vmargins hmargin...",ARTS
1216,49777184,MANAGEMENT CONSULTANT Summary ...,"<div class=""fontsize fontface vmargins hmargin...",CONSULTANT
2305,22848179,DIRECTOR OF COMMUNITY Ski...,"<div class=""fontsize fontface vmargins hmargin...",ARTS
...,...,...,...,...
2072,24710433,"MARKETING, COMMNICATIONS, & PUBLIC RE...","<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS
473,22391901,GRADATION CANDIDATE ADVOCATE Ex...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE
1511,37931076,FINANCE ASSISTANT Professional ...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE
923,14140903,GRADUATE RESEARCH ASSISTANT Pro...,"<div class=""fontsize fontface vmargins hmargin...",AGRICULTURE


In [8]:
cats[0]

'HR'

In [9]:
cats

['HR',
 'DESIGNER',
 'INFORMATION-TECHNOLOGY',
 'TEACHER',
 'ADVOCATE',
 'BUSINESS-DEVELOPMENT',
 'HEALTHCARE',
 'FITNESS',
 'AGRICULTURE',
 'BPO',
 'SALES',
 'CONSULTANT',
 'DIGITAL-MEDIA',
 'AUTOMOBILE',
 'CHEF',
 'FINANCE',
 'APPAREL',
 'ENGINEERING',
 'ACCOUNTANT',
 'CONSTRUCTION',
 'PUBLIC-RELATIONS',
 'BANKING',
 'ARTS',
 'AVIATION']

In [10]:
y = []
for index, row in df.iterrows():
    r = []
    for j in range(categories_l):
        if(row['Category'] == cats[j]):
            r.append(1)
        else:
            r.append(0)
    y.append(r)
    


In [11]:
y = np.array(y).astype(np.float64)

In [12]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [13]:
X = (df["Resume_str"].str.lower().replace('\s+', ' ').replace('\n', ' ')).values

In [14]:
# Example to ensure all entries are strings (Python way, not the most efficient for large data)
X = [str(entry) for entry in X]

In [15]:
X = np.array(X)

In [16]:
vocab_size = 1500
embedding_dim = 32
max_length = 1000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


In [17]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

In [18]:
y[55]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0.])

In [19]:
cats[np.argmax(y[55])]

'ENGINEERING'

In [20]:
type(X)

numpy.ndarray

In [21]:
X_seq = tokenizer.texts_to_sequences(X)
X_pad = np.array(pad_sequences(X_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type))

In [22]:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='sigmoid'),
    #tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(categories_l, activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
#model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam')
#ops = tf.keras.optimizers.Adagrad(
  #  learning_rate=1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(X_pad, y, epochs=500, batch_size=32, validation_split=0.25, verbose=2)

Epoch 1/500


I0000 00:00:1714854147.312257     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


59/59 - 9s - 152ms/step - accuracy: 0.0456 - loss: 3.2660 - val_accuracy: 0.0451 - val_loss: 3.1823
Epoch 2/500
59/59 - 0s - 3ms/step - accuracy: 0.0521 - loss: 3.1825 - val_accuracy: 0.0515 - val_loss: 3.1510
Epoch 3/500
59/59 - 0s - 3ms/step - accuracy: 0.0429 - loss: 3.1707 - val_accuracy: 0.0419 - val_loss: 3.1404
Epoch 4/500
59/59 - 0s - 3ms/step - accuracy: 0.0510 - loss: 3.1522 - val_accuracy: 0.0548 - val_loss: 3.1366
Epoch 5/500
59/59 - 0s - 3ms/step - accuracy: 0.0515 - loss: 3.1416 - val_accuracy: 0.0467 - val_loss: 3.1325
Epoch 6/500
59/59 - 0s - 3ms/step - accuracy: 0.0451 - loss: 3.1415 - val_accuracy: 0.0580 - val_loss: 3.1240
Epoch 7/500
59/59 - 0s - 3ms/step - accuracy: 0.0660 - loss: 3.1234 - val_accuracy: 0.0789 - val_loss: 3.1148
Epoch 8/500
59/59 - 0s - 3ms/step - accuracy: 0.0725 - loss: 3.1141 - val_accuracy: 0.0918 - val_loss: 3.1009
Epoch 9/500
59/59 - 0s - 3ms/step - accuracy: 0.0998 - loss: 3.0872 - val_accuracy: 0.1079 - val_loss: 3.0727
Epoch 10/500
59/59 -

In [24]:
sentence = ["""Education
Experience
University of Guelph Robotics Team Software Member Sep. 2023 – Present
University of Guelph Guelph, ON
Experience with using WSL and Linux to navigate robotics workspaces and programs
Python programming to control robot simulations, image processing for simulation camera output
Image augmentation with opencv and soon tensorflow and onnx for shrub detection(ongoing)
Introduced to using docker for multi-platform deployment of robotics code
Guelph AI Club Technical Lead and Executive Jan. 2024 – Present
University of Guelph Guelph, ON
Lead a team to create an AI assistant similar to Jarvis from Iron Man
"""]
sentence[0] = sentence[0].lower().replace('\s+', ' ').replace('\n', ' ')

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
p = model.predict(padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


In [25]:
cats[np.argmax(p)]

'ARTS'

In [26]:
np.argmax(p)

22

In [27]:
p

array([[2.31086336e-13, 1.28222525e-03, 5.49170409e-06, 1.54403061e-01,
        1.05751342e-05, 3.09369843e-02, 9.02248303e-06, 3.19272161e-01,
        1.70429330e-02, 6.64281892e-04, 2.20992770e-02, 3.64579205e-06,
        3.10360231e-02, 1.57663366e-03, 1.11207024e-04, 2.98879588e-10,
        2.67575681e-03, 8.98342696e-04, 4.40231049e-13, 2.36005581e-05,
        3.89674959e-10, 8.84936435e-06, 4.16539490e-01, 1.40038203e-03]],
      dtype=float32)

In [28]:
np.max(p)

0.4165395

In [29]:
model.save('model.h5')

In [30]:
indices = np.argsort(p)[0][::-1]

In [31]:
indices

array([22,  7,  3, 12,  5, 10,  8, 16, 13, 23,  1, 17,  9, 14, 19,  4,  6,
       21,  2, 11, 20, 15, 18,  0])

In [32]:
indices[0]

22

In [33]:
indices

array([22,  7,  3, 12,  5, 10,  8, 16, 13, 23,  1, 17,  9, 14, 19,  4,  6,
       21,  2, 11, 20, 15, 18,  0])

In [34]:
for i in range(0, 3):
    print(cats[indices[i]])
    print(p[0][indices[i]])

ARTS
0.4165395
FITNESS
0.31927216
TEACHER
0.15440306


In [35]:
cats[0]

'HR'

In [36]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

from IPython.display import FileLink
FileLink('tokenizer.pickle')

/kaggle/working/tokenizer.pickle